In [47]:
import pickle
from keras.layers import Dense, Embedding, SimpleRNN, GlobalMaxPooling1D, Conv1D, Dropout, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

num_words = 10000
max_review_len = 100
csv_path = '../build/data3.csv'
model_save_path = '../build/best_model.h5'
out_path = '../output/'

def lstm_model():
    model = Sequential()
    model.add(Embedding(num_words, 64, input_length=max_review_len))
    model.add(LSTM(128))
    model.add(Dense(1, activation='sigmoid'))
    return model
    
def rec_model():
    model = Sequential()
    model.add(Embedding(num_words, 2, input_length=max_review_len))
    model.add(SimpleRNN(8))
    model.add(Dense(1, activation='sigmoid'))
    return model

def conv_model():
    model = Sequential()
    model.add(Embedding(num_words, 64, input_length=max_review_len))
    model.add(Conv1D(250, 5, padding='valid', activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

class Type:
    def __init__(self, type_name, model):
        self.model_path = out_path + type_name + '/best_model.h5'
        self.tokenizer_path = out_path + type_name + '/tokenizer.pickle'
        self.out_path = out_path + type_name + '/output'
        self.model = model
    
lstm = Type("lstm", lstm_model())
reccurent = Type("rec", rec_model())
conv = Type("conv", conv_model())


net_type = lstm
model = net_type.model
model.load_weights(net_type.model_path)
with open(net_type.tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)

In [48]:
text = "Out of all the reviews I wrote This one is probably the most serious one I wrote. For starters the community of this game sucks just like every online game You don't wanna talk to them because they're all just jerks who wanna say you suck or you camped or blah blah blah its rare to get a nice compliment out of people in this game.But you don't have to talk to the community either. Unlike most online games this one doesn't have a mic system so you don't have to worry about hearing a twelve year old cry baby raging at you for nothing. Only time you can talk to people in the game is in the beginning before the match starts if your a survivor and at the end after the match is over as both killer and survivor The end chat is the only one the killer can talk to everyone in the match though"
sequence = tokenizer.texts_to_sequences([text])
data = pad_sequences(sequence, maxlen=max_review_len)
result = model.predict(data)
result

array([[0.9274228]], dtype=float32)

In [49]:
print("NO" if result < 0.5 else "YES")

YES


In [50]:
input_path = '../build/input'
f = open(input_path)
inp = f.read().split('\n')
f.close()
inp

["Out of all the reviews I wrote This one is probably the most serious one I wrote. For starters the community of this game sucks just like every online game You don't wanna talk to them because they're all just jerks who wanna say you suck or you camped or blah blah blah its rare to get a nice compliment out of people in this game.But you don't have to talk to the community either. Unlike most online games this one doesn't have a mic system so you don't have to worry about hearing a twelve year old cry baby raging at you for nothing. Only time you can talk to people in the game is in the beginning before the match starts if your a survivor and at the end after the match is over as both killer and survivor The end chat is the only one the killer can talk to everyone in the match though",
 'My love',
 'I do not like']

In [51]:
sequence = tokenizer.texts_to_sequences(inp)
data = pad_sequences(sequence, maxlen=max_review_len)
result = model.predict(data)
result

array([[0.9274228],
       [0.4116594],
       [0.6238719]], dtype=float32)

In [52]:
#result = [1,0,1]
output_path = net_type.out_path
f = open(output_path, 'w')
s = "\n".join(map(str, result))
f.write(s)
f.close()
s

'[0.9274228]\n[0.4116594]\n[0.6238719]'